In [ ]:
pip install opencv-python-headless


In [ ]:
pip install albumentations

In [ ]:
pip install tqdm

In [ ]:
import os
import pandas as pd
import cv2
import albumentations as A
from tqdm import tqdm

# Load metadata
metadata_path = "data/data-1729656846556.csv"  # Path to the original metadata file
metadata = pd.read_csv(metadata_path)

# List to store the new metadata
new_metadata = []

# Augmentation settings
augment = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Rotate(limit=15, p=0.3),
    A.ToGray(p=0.2),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
    A.Blur(blur_limit=3, p=0.2),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=20, p=0.3)
])

# Filter and augment Declined images
for _, row in tqdm(metadata.iterrows(), total=len(metadata)):
    if row['image_status'] == "Declined":
        original_image_path = os.path.join("data/cropdata", str(row['job_no']), row['image_name'])
        
        # Load image
        image = cv2.imread(original_image_path)
        if image is None:
            continue
        
        # Apply augmentation
        augmented_image = augment(image=image)['image']
        
        # Create filename and path for the augmented image (add _augmented)
        original_folder = os.path.join("data/cropdata", str(row['job_no']))
        new_image_name = f"{os.path.splitext(row['image_name'])[0]}_augmented.jpg"
        new_image_path = os.path.join(original_folder, new_image_name)
        
        # Save the augmented image
        cv2.imwrite(new_image_path, augmented_image)
        
        # Copy and modify the original metadata
        new_row = row.copy()
        new_row['image_name'] = new_image_name  # Change to augmented image filename
        new_row['image_path'] = new_image_path  # Add new path
        new_metadata.append(new_row)

    # Add the original data to the new metadata as is
    original_row = row.copy()
    original_image_path = os.path.join("data/cropdata", str(row['job_no']), row['image_name'])
    original_row['image_path'] = original_image_path  # Add the original path
    new_metadata.append(original_row)

# Save the new metadata file
new_metadata_df = pd.DataFrame(new_metadata)
new_metadata_df.to_csv("data/augmented_metadata.csv", index=False)

